In [ ]:
#vlc Integrated with model
import numpy as np
from keras.models import model_from_json
import operator
import cv2
import sys
import os
import vlc
import time
json_file = open("mymodel/model-bw.json", "r")
model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(model_json)
loaded_model.load_weights("model-bw.h5")
print("Loaded model from disk")
cap = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_SIMPLEX 
player = vlc.MediaPlayer("Input Your song directory here/songName.mp3")
#categories = {0: '2fingure', 1: 'fist',2: 'no gestures' ,3: 'palm', 4: 'yo'}
while True:
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)
    cv2.rectangle(frame, (400,30),(620,350),(0,0,255),2)
    roi=frame[30:350,400:620]
    roi = cv2.resize(roi, (64,64))
    hsv=cv2.cvtColor(roi,cv2.COLOR_BGR2HSV)
    lower_skin = np.array([0, 30, 60 ], dtype=np.uint8)
    upper_skin = np.array([20, 150, 255], dtype=np.uint8)
    mask = cv2.inRange(hsv, lower_skin, upper_skin)
    kernel=np.ones((3,3),dtype=np.uint8)
    mask = cv2.dilate(mask,kernel,iterations = 4)
    mask = cv2.GaussianBlur(mask,(15,15),100)
    cv2.imshow("test", mask)    
    result = loaded_model.predict(mask.reshape(1, 64,64, 1))
    prediction = {'2fingure': result[0][0], 
                  'fist': result[0][1], 
                  'palm': result[0][3],
                  'yo': result[0][4],
                  'no gesture':result[0][2]}
    prediction = sorted(prediction.items(), key=operator.itemgetter(1), reverse=True)
    cv2.putText(frame, prediction[0][0], (10, 120), cv2.FONT_HERSHEY_PLAIN, 1, (0,255,255), 1)    
    if result[0][2]:
        #print("No Gesture")
        cv2.putText(frame, 'No Action', (5, 50), font, 0.7, (0, 255, 255), 2, cv2.LINE_AA)
    elif result[0][0]:
        #print("2fingures")
        player.play()
        cv2.putText(frame, 'Play', (5, 50), font, 0.7, (0, 255, 255), 2, cv2.LINE_AA)
    elif result[0][1]:
        #print("fist")
        player.stop()
        cv2.putText(frame, 'Pause', (5, 50), font, 0.7, (0, 255, 255), 2, cv2.LINE_AA)
    elif result[0][3]:
        #print("palm")
        a = player.audio_get_volume()
        if(a+5<=100):
                player.audio_set_volume(a+5)
                #time.sleep(0.5)
        cv2.putText(frame, 'Vol Up', (5, 50), font, 0.7, (0, 255, 255), 2, cv2.LINE_AA) 
    elif result[0][4]:
        #print("yo")
        a = player.audio_get_volume()
        if(a-5>=0):
                player.audio_set_volume(a-5)
                #time.sleep(0.5)
        cv2.putText(frame, 'Vol Down', (5, 50), font, 0.7, (0, 255, 255), 2, cv2.LINE_AA)
    cv2.imshow("Frame", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        player.stop()
        break
cap.release()
cv2.destroyAllWindows()